In [1]:
using CSV
using Dates
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-03-01_one_column.csv"))

,Column1,hospital_name,ccn,collection_week,fips_code,hospitalization_percentage,ICU_percentages,hospitalization_2021-03-02,target_date,hospitalizations
,Int64,String,String?,Date,Float64,Float64?,Float64?,Float64?,Date,Float64?
1,10,PRATTVILLE BAPTIST HOSPITAL,010108,2021-02-19,1001.0,1.0,0.21875,1.70004,2021-03-01,missing
2,11,NORTH BALDWIN INFIRMARY,010129,2021-02-19,1003.0,0.160221,0.0526316,0.655209,2021-03-01,missing
3,12,SOUTH BALDWIN REGIONAL MEDICAL CENTER,010083,2021-02-19,1003.0,0.329912,0.370787,1.34915,2021-03-01,missing
4,13,THOMAS HOSPITAL,010100,2021-02-19,1003.0,0.509867,0.337209,2.08506,2021-03-01,missing
5,14,MEDICAL CENTER BARBOUR,010069,2021-02-19,1005.0,1.0,0.684211,0.505463,2021-03-01,missing
6,15,BIBB MEDICAL CENTER,010058,2021-02-19,1007.0,0.833333,0.0,0.320396,2021-03-01,missing
7,16,ST VINCENTS BLOUNT,011305,2021-02-19,1009.0,0.966667,0.2,0.920051,2021-03-01,missing
8,17,BULLOCK COUNTY HOSPITAL,010110,2021-02-19,1011.0,0.0666667,0.0,0.0123677,2021-03-01,missing
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,010150,2021-02-19,1013.0,0.966667,0.0,0.326176,2021-03-01,missing


In [4]:
function firstval(xs)
    isbad(x) = isnothing(x) || ismissing(x)
    if all(isbad.(xs))
        return missing
    else
        return xs[findfirst(x -> !isbad(x), xs)]
    end
end;

In [5]:
ccn_lookup_df = combine(groupby(rawdata, [:Column1, :hospital_name]), :ccn => firstval => :ccn)
ccn_lookup = Dict(row.Column1 => row.ccn for row in eachrow(ccn_lookup_df));

In [6]:
filter!(row -> ismissing(row["hospitalization_2021-03-02"]), rawdata);
filter!(row -> row.target_date != Date(2021, 03, 01), rawdata);

In [7]:
data = select(rawdata,
    :hospital_name => :hospital,
    :Column1 => ByRow(x -> ccn_lookup[x]) => :hospital_id,
    :target_date => :date,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*coalesce(y,0.3)) => :admitted_moderate_icu,
    ["hospitalizations", "ICU_percentages"] => ByRow((x,y) -> x*(1.0-coalesce(y,0.3))) => :admitted_moderate_acute,
    "hospitalizations" => :admitted_moderate_allbeds,
)
sort!(data, [:hospital, :hospital_id, :date])

,hospital,hospital_id,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,String?,Date,Float64?,Float64?,Float64?
1,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-03,0.0,0.309348,0.309348
2,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-04,0.0,0.305027,0.305027
3,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-05,0.0,0.295876,0.295876
4,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-06,0.0,0.294351,0.294351
5,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-07,0.0,0.259991,0.259991
6,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-08,0.0,0.266955,0.266955
7,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-09,0.0,0.253955,0.253955
8,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-10,0.0,0.243277,0.243277
9,ABBEVILLE AREA MEDICAL CENTER,421301,2021-03-11,0.0,0.240259,0.240259


In [8]:
data |> CSV.write("../data/hhs_forecast_2021_03_01.csv")

"../data/hhs_forecast_2021_03_01.csv"